In [1]:
# reloading modules if needed
from importlib import reload

# first import the whole package
import PyAPD

# then load all the functions in the .py files in the .../PyAPD/ directory:
from PyAPD import *

torch.set_printoptions(precision=10) # show more digits

device = "cuda" if torch.cuda.is_available() else "cpu"
dt = torch.float64
torch.set_default_dtype(dt)
#torch.set_default_device(device)

import copy

import scipy
import scipy.stats
dist_volumes = getattr(scipy.stats, 'lognorm')
arg = 1.0
loc = 0.5
scale = 1.0

In [2]:

# seed = 5 #fix seed
# dist_volumes = getattr(scipy.stats, 'lognorm')
# # draw a sample of volumes (to be "normalised" later)
# arg = 1.0
# loc = 0.5
# scale = 1.0

# np.random.seed(seed=seed)
# TV = dist_volumes.rvs(arg, loc=loc, scale=scale,size=200)
# plt.hist(TV,bins = 'auto',density=True)

# x_range = np.linspace(1e-5, np.max(TV), 200)

# pdf_volumes_fitted = dist_volumes.pdf(x_range, arg, loc=loc, scale=scale)
# plt.plot(x_range,pdf_volumes_fitted)
# #pdf_volumes_fitted = dist_volumes.pdf(x_range, 1.0, loc=0.0, scale=2.0)
# #plt.plot(x_range,pdf_volumes_fitted)
# 

# N = 10
# np.random.seed(seed=seed)
# TV = dist_volumes.rvs(arg, loc=loc, scale=scale,size=N)
# TV = torch.from_numpy(TV).to(device,dtype=dt)
# TV

# TV.sum()

In [3]:
#multi or single phase
mp = False

# D = 2  # Dimension of the ambient space
# #Ms = [1200]
# Ns = [25,50,100,250,500,1000,2500,5000]

D = 3  # Dimension of the ambient space
Ms = [200]
Ns = [50,100,250,500,1000,2500,5000,10000]

roe = 0.2

apd = apd_system(N=100, D=D,dt=torch.float64,
                  pixel_size_prefactor = 2,
                  ani_thres = 0.3,
                  radius_of_exclusion = roe,
                  heuristic_W = False)

file_name = f'optimal_apd_{D}D_{"mp" if mp else "sp"}_{roe}.pkl'

print(file_name)

dts = [torch.float64] # single precision or double precision
anisotropies = [0.0, 0.3, 0.7]
seeds = [i for i in range(10)]

total_no_data_points = len(Ns)*len(anisotropies)*len(seeds)
print(*Ns)
print(*dts)
print(*anisotropies)
print(*seeds)

optimal_apd_3D_sp_0.2.pkl
50 100 250 500 1000 2500 5000 10000
torch.float64
0.0 0.3 0.7
0 1 2 3 4 5 6 7 8 9


In [5]:
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("FAKE FIRST RUN")

kk=-1
for seed in seeds[:1]:
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("Random seed =", seed)
    apd.seed = seed
    for dt in dts[:1]:
        torch.set_default_dtype(dt)
        apd.dt = dt        
        for ani_thres in anisotropies[:1]:
            apd.ani_thres = ani_thres
            for N in Ns[:1]:
                print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
                print("N = ", N, "seed = ", seed, "ani_thres = ", ani_thres)
                kk+= 1
                apd.optimality = False
                apd.N = N
                
                ### SAMPLING
                # to ensure consistenscy, we want to do the sampling in single precision
                torch.set_default_dtype(torch.float32)
                
                ### sampling seed points
                torch.manual_seed(seed)
                X = sample_seeds_with_exclusion(N,dim=D,radius_prefactor = apd.radius_of_exclusion)
                
                ### sampling anisotropy matrices
                torch.manual_seed(seed+10)
                As = sample_normalised_spd_matrices(N,dim = D, ani_thres = ani_thres)
                
                # ### sampling target masses (optional)
                if mp:
                    np.random.seed(seed=seed + 20)
                    target_masses = dist_volumes.rvs(arg, loc=loc, scale=scale,size=N)
                    target_masses = torch.from_numpy(target_masses/target_masses.sum()).to(device,dtype=dt)
                
                ### back to the precision we actually want to work with
                ### (when populating the class, the conversion will take place automatically):
                torch.set_default_dtype(apd.dt)
                apd.set_X(X=X)
                apd.set_As(As=As)
                if mp:
                    apd.set_target_masses(target_masses = target_masses)
                else:
                    apd.set_target_masses()
                apd.set_W()
                #apd.set_pixel_params(pixel_params = (1200,)*apd.D)
                apd.set_pixel_params()
                apd.assemble_pixels()
                #apd.assemble_apd(verbose=True,record_time = True)
                apd.find_optimal_W(record_time = True,max_iter = 2000)
                #apd.data[kk] = copy.copy(apd.__dict__)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
FAKE FIRST RUN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Random seed = 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
N =  50 seed =  0 ani_thres =  0.0
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0002000000, device='cuda:0')
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.043174
         Iterations: 16
         Function evaluations: 17
It took 0.032912492752075195 seconds to find optimal W.


In [6]:
counter = 0
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print("ACTUAL RUN")
kk=-1
for seed in seeds:
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("Random seed =", seed)
    apd.seed = seed
    for dt in dts:
        torch.set_default_dtype(dt)
        apd.dt = dt        
        for ani_thres in anisotropies:
            apd.ani_thres = ani_thres
            for N in Ns:
                print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
                print("N = ", N, "seed = ", seed, "ani_thres = ", ani_thres)
                kk+= 1
                print("Populating data point",kk+1, " / ", total_no_data_points)
                apd.optimality = False
                apd.N = N
                
                ### SAMPLING
                # to ensure consistenscy, we want to do the sampling in single precision
                torch.set_default_dtype(torch.float32)
                
                ### sampling seed points
                torch.manual_seed(seed)
                X = sample_seeds_with_exclusion(N,dim=D,radius_prefactor = apd.radius_of_exclusion)
                
                ### sampling anisotropy matrices
                torch.manual_seed(seed+10)
                As = sample_normalised_spd_matrices(N,dim = D, ani_thres = ani_thres)
                
                ### sampling target masses (optional)
                if mp:
                    np.random.seed(seed=seed+20)
                    target_masses = dist_volumes.rvs(arg, loc=loc, scale=scale,size=N)
                    target_masses = torch.from_numpy(target_masses/target_masses.sum()).to(device,dtype=dt)

                ### back to the precision we actually want to work with
                ### (when populating the class, the conversion will take place automatically):
                torch.set_default_dtype(apd.dt)
                apd.set_X(X=X)
                apd.set_As(As=As)
                if mp:
                    apd.set_target_masses(target_masses = target_masses)
                else:
                    apd.set_target_masses()
                apd.set_W()
                #apd.set_pixel_params(pixel_params = (1200,)*apd.D)
                apd.set_pixel_params(verbose=True)
                apd.assemble_pixels()
                #apd.assemble_apd(verbose=True,record_time = True)
                apd.find_optimal_W(record_time = True, max_iter = 2000)
                apd.check_optimality()
                if apd.optimality == False:
                    counter += 1
                apd.Y = None
                apd.PS = None
                apd.y = None
                apd.a = None
                apd.x = None
                #apd.W = None
                apd.w = None
                
                d = apd.__dict__
                apd.data[kk] = copy.copy({i:d[i] for i in d if i!='data'})

import pickle

with open(file_name, "wb") as fp:   #Pickling
     pickle.dump(apd, fp)

# import bz2
# import pickle
# import _pickle as cPickle

# def compressed_pickle(title, data):
#     with bz2.BZ2File(title + '.pbz2', 'w') as f: 
#         cPickle.dump(data, f)
# compressed_pickle("test",apd.data)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ACTUAL RUN
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Random seed = 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
N =  50 seed =  0 ani_thres =  0.0
Populating data point 1  /  240
M =  34
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0002000000, device='cuda:0')
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.043174
         Iterations: 16
         Function evaluations: 17
It took 0.03307819366455078 seconds to find optimal W.
The APD is optimal!
Percentage error =  0.625890494606006
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
N =  100 seed =  0 ani_thres =  0.0
Populating data point 2  /  240
M =  42
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0001000000, device='cuda:0')
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current func

In [6]:
counter

1